In [ ]:
# import libraries
import torch
import os
import pandas
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataset import Dataset
from torchvision.models import resnet50
from torchvision.models.resnet import ResNet50_Weights